In [3]:
import tensorflow as tf

In [117]:
wordEmbeddingDimension = 50
vocabularySize=100
labels=10
filterSizes_paragraph = [3]
filterSizes_allPara=3
paragraphLength=10
num_filters_parargaph=15
num_filters_allPara=20
maxParagraph = 5

filterShapeOfAllPara =[filterSizes_allPara,3,1,num_filters_allPara]
fullyConnectedLayerInput = 150


In [120]:
wordEmbedding = tf.Variable(tf.random_uniform([vocabularySize, wordEmbeddingDimension], -1.0, 1.0))

# paragraphWords = tf.placeholder(tf.int32,[paragraphLength])

paragraphList = []
for i in range(maxParagraph):
    paragraphList.append(tf.placeholder(tf.int32,[paragraphLength]))

target = tf.placeholder(tf.float32,[labels])

In [32]:
paragraph=[[1,2,3,4,5,6,7,8,9,10],[1,2,3,4,5,6,7,8,9,10]]


In [33]:
def getParagraphEmbedding(paragraphWords):
    paraEmbedding=tf.nn.embedding_lookup(wordEmbedding,paragraphWords)
    
    return tf.expand_dims(tf.expand_dims(paraEmbedding, -1),0)

In [34]:
paragraphVector = getParagraphEmbedding(paragraphList[0])

In [26]:
paragraphVector

<tf.Tensor 'ExpandDims_11:0' shape=(1, 10, 50, 1) dtype=float32>

In [43]:
def convLayeronParagraph(paragraphVector,filterSizes,num_input_channels,num_filters):
    
    pooled_outputs=[]
    for filter_size in filterSizes:
        shape = [filter_size,wordEmbeddingDimension,1,num_filters]
        
        weights = tf.Variable(tf.truncated_normal(shape, stddev=0.1))
        bias= tf.Variable(tf.constant(0.1, shape=[num_filters]))
        conv = tf.nn.conv2d(
                    paragraphVector,
                    weights,
                    strides=[1, 1, wordEmbeddingDimension, 1],
                    padding="SAME",
                    name="conv")
        
        h = tf.nn.relu(tf.nn.bias_add(conv, bias), name="relu")
        pool_length=5
        pooled = tf.nn.max_pool(
                    h,
                    ksize=[1, pool_length, 1, 1],
                    strides=[1, pool_length, 1, 1],
                    padding='SAME',
                    name="pool")
        pooled_outputs.append(pooled)
    return tf.reshape(tf.concat(pooled_outputs,axis=0),[1,-1])
#     filter_size = 

In [44]:
paragraphCNNOutput = convLayeronParagraph (paragraphVector,filterSizes,1,num_filters_parargaph)

In [45]:
paragraphCNNOutput

<tf.Tensor 'Reshape_21:0' shape=(1, 30) dtype=float32>

In [97]:
def convLayerCombineParagraph(paragraphVectorList,filterSizes_paragraph,filterShapeOfAllPara,num_filters_parargaph,num_filters_allPara):
    
    paragraphCNNEmbedding=[]
    
    for paragraph in paragraphVectorList:
        paragraphVector = getParagraphEmbedding(paragraph)
        cnnEmbedding = convLayeronParagraph(paragraphVector,filterSizes_paragraph,1,num_filters_parargaph)
        paragraphCNNEmbedding.append(cnnEmbedding)
    
    allParagraph=tf.expand_dims(tf.expand_dims(tf.concat(paragraphCNNEmbedding,axis=0),-1),0)
    
    shape = filterShapeOfAllPara
    
    weights= tf.Variable(tf.truncated_normal(shape, stddev=0.1))
    bias= tf.Variable(tf.constant(0.1, shape=[num_filters_allPara]))
    
    conv = tf.nn.conv2d(
                    allParagraph,
                    weights,
                    strides=[1, 1, 1, 1],
                    padding="SAME",
                    name="conv")
    h = tf.nn.relu(tf.nn.bias_add(conv, bias), name="relu")
    return tf.reshape(allParagraph,[1,-1])
    

In [98]:
convOutput=convLayerCombineParagraph(paragraphList,filterSizes_paragraph,filterShapeOfAllPara,num_filters_parargaph,num_filters_allPara)
print(convOutput)

Tensor("Reshape_94:0", shape=(1, 150), dtype=float32)


In [118]:
def fullyConnectedLayer(convOutput,labels):
    shape = [fullyConnectedLayerInput,labels]
    weights =tf.Variable(tf.truncated_normal(shape, stddev=0.1))
    bias = tf.Variable(tf.constant(0.1, shape=[labels]))
    layer = tf.nn.sigmoid(tf.matmul(convOutput, weights) + bias)
    return layer

In [121]:
prediction=fullyConnectedLayer(convOutput,labels)


In [122]:
cross_entropy = -tf.reduce_sum(((target*tf.log(prediction + 1e-9)) + ((1-target) * tf.log(1 - prediction + 1e-9)) )  , name='xentropy' ) 
cost = tf.reduce_mean(cross_entropy)
optimizer = tf.train.AdamOptimizer(learning_rate=1e-4).minimize(cost)